In [4]:
import sys
import os
import importlib
import pandas as pd
import yfinance as yf
from typing import Callable

# add utils path
src_path = os.path.abspath('../../')
if src_path not in sys.path:
    sys.path.append(src_path)

# import custom functions
from src.investing_strategies import pca_fa

# reload module on change
importlib.reload(sys.modules['src.investing_strategies'])

<module 'src.investing_strategies' from 'c:\\Users\\Yang\\Documents\\Projects\\funance_blog\\src\\investing_strategies\\__init__.py'>

In [5]:
# get data
PATH = '../data/dow30.csv'
tickers = pd.read_csv(PATH)['ticker'].tolist()

factors_df = pd.read_csv('../data/fama_french_momentum_daily.csv')
factors_df['Date'] = pd.to_datetime(factors_df['Date'])
factors_df.set_index('Date', inplace=True)
factors_df = factors_df / 100 # convert to decimals

prices = yf.download(tickers, start='2000-01-01', end='2024-06-30')['Adj Close']
returns = prices.pct_change(fill_method=None)[1:]

[*********************100%%**********************]  30 of 30 completed


In [6]:
df = returns.loc['2018-12-31':'2023-12-31']
pca_fa(df, factors_df, 2)

c:\Users\Yang\Documents\Projects\funance_blog\venv\lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


{'GS': 0.1539, 'AAPL': 0.2497, 'MSFT': 0.1578, 'TRV': 0.195, 'CAT': 0.2436}

In [ ]:
def backtest_strategy(
    strategy: Callable,
    historical_data: pd.DataFrame,
    trading_frequency: str='monthly',
)